# Creating choropleth (i.e. heat) maps

In [1]:
import io
import requests

import zipfile
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

from bokeh.io import show, output_file, output_notebook, export_png
from bokeh.models import ColumnDataSource, HoverTool, PanTool, WheelZoomTool, LinearColorMapper, BasicTicker, FixedTicker, ColorBar
from bokeh.palettes import brewer 
from bokeh.plotting import figure, save
from bokeh.transform import transform
from bokeh.models.glyphs import Patches

output_notebook() 

Loading BokehJS ...

## I. Load ACS data

In [2]:
## Median rental price data (as a % of household income) for all counties in New York City
## Downloaded manually from https://factfinder.census.gov/ 
## There seems to be a pretty neat API someone wrote (https://jtleider.github.io/censusdata/index.html) 
## to download ACS data non-manually, but I tried it out and unfortunately the geographic identifiers 
## were really messy and not worth the effort to clean

df = pd.read_csv("ACS_16_5YR_B25071_with_ann.csv")
df.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01
0,Id,Id2,Geography,Estimate; Median gross rent as a percentage of...,Margin of Error; Median gross rent as a percen...
1,1500000US360050001000,360050001000,"Block Group 0, Census Tract 1, Bronx County, N...",-,**
2,1500000US360050001001,360050001001,"Block Group 1, Census Tract 1, Bronx County, N...",-,**
3,1500000US360050002000,360050002000,"Block Group 0, Census Tract 2, Bronx County, N...",-,**
4,1500000US360050002001,360050002001,"Block Group 1, Census Tract 2, Bronx County, N...",50.0+,***


In [3]:
## Do some cleaning
df['id_string'] = df['GEO.id2'].astype('str')
df.drop(df.index[0], inplace=True)

In [4]:
def clean_estimate(estimate):
    if estimate == '50.0+': 
        return 50
    elif estimate == '10.0-':
        return 10
    elif estimate == '-':
        return(np.nan)
    else: 
        return(estimate)

In [5]:
df['estimate_clean'] = df.apply(lambda row: clean_estimate(row['HD01_VD01']), axis=1)

In [6]:
df['estimate_float'] = df['estimate_clean'].astype('float')

## II. Load shapefiles from U.S. Census

In [7]:
url = "https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2017&layergroup=Block+Groups/tl_2017_36_bg.zip"
request_zip = requests.get(url)
file_zip = zipfile.ZipFile(io.BytesIO(request_zip.content))
file_zip.extractall()
shp = gpd.read_file("tl_2017_36_bg/tl_2017_36_bg.shp")

In [9]:
## Some shapefiles contain multipolygons, which make life harder later on. This cell checks whether or not there are
## multipolygons, so that we can either remove those observations or expand the multipolygons

multipoly = []
for idx, row in shp.iterrows():
    if type(row.geometry) == MultiPolygon:
        multipoly.append(row.GEOID)
multipoly

['360450604005',
 '360450603004',
 '360610001001',
 '360610002022',
 '360050516005',
 '360450617002',
 '360450603002',
 '360810918000',
 '360810938000',
 '360810954000',
 '360610152006',
 '360050504001',
 '360610009002',
 '360610062002',
 '360610086011',
 '360450603003',
 '360470702031',
 '360359708001',
 '360810928000',
 '360610005002',
 '360811072021',
 '360650249001',
 '360810922000',
 '360010146121',
 '360050019005',
 '361031907053']

In [10]:
## I chose not to be lazy and to expand them instead of deleting

shp_expanded = shp.set_index(['GEOID'])['geometry'].apply(pd.Series).stack().reset_index()
shp_expanded.rename(columns = {0: 'geometry'}, inplace = True)
shp = shp.drop(columns = 'geometry')
shp = shp_expanded.merge(shp, on = 'GEOID', how = 'left')

In [11]:
## Convert crs to lat/lon for plotting

shp_clean = gpd.GeoDataFrame(shp, crs={'init': 'epsg:4326'}, geometry=shp['geometry'])

In [12]:
## Plotting shapes (or "patches") in Bokeh requires separate lists (of lists) of the x and y coordinates of each 
## polygon, so this function extracts those lists from the shapefile's geometry column

def get_coords(row, geom, coord_type, shape_type):
    """
    Returns the coordinates ('x' or 'y') of edges of a Polygon exterior.
    
    :param: (GeoPandas Series) row : The row of each of the GeoPandas DataFrame.
    :param: (str) geom : The column name.
    :param: (str) coord_type : Whether it's 'x' or 'y' coordinate.
    :param: (str) shape_type
    """
    # Parse the exterior of the coordinate
    if shape_type == 'polygon':
        exterior = row[geom].exterior
        if coord_type == 'x':
            return list( exterior.coords.xy[0] )    
        
        elif coord_type == 'y':
            return list( exterior.coords.xy[1] )

In [13]:
shp_clean['x'] = shp_clean.apply(get_coords, 
                                 geom='geometry', 
                                 coord_type='x', 
                                 shape_type='polygon',
                                 axis=1)
                                 
shp_clean['y'] = shp_clean.apply(get_coords, 
                                 geom='geometry', 
                                 coord_type='y', 
                                 shape_type='polygon',
                                 axis=1)

## III. Merge shapefiles onto ACS

In [14]:
to_plot = df.merge(shp_clean, 'left', left_on='id_string', right_on = 'GEOID')

In [15]:
## Things are getting a little messy, so I'm getting rid of the variables that won't be used

to_plot = to_plot[['id_string','estimate_float', 'x', 'y']]

## IV. Make map!

In [16]:
colors = brewer['OrRd'][9]
mapper = LinearColorMapper(palette=colors,
                          low=50, 
                          high=10, 
                          nan_color='#EAE8E7'
                         )

source = ColumnDataSource(data=dict(
                                    x=to_plot['x'], 
                                    y=to_plot['y'],
                                    color=to_plot['estimate_float']
                                    )
                         )

p = figure(title="NYC Median Rent (% of Household Income)",
            x_axis_location=None, 
            y_axis_location=None,
            plot_width=1000, 
            plot_height=700
          )

p.grid.grid_line_color = None
p.outline_line_color = None
p.title.text_font = "nunito"
p.title.align = "center"
p.title.text_font_size="40px"

p.patches('x', 
          'y', 
          source=source, 
          fill_color= transform('color', mapper),
          fill_alpha=0.8, 
          line_color="black", 
          line_width=0.3)

ticker = FixedTicker(ticks=[10,20,30,40,50])

color_bar = ColorBar(color_mapper=mapper, 
                     ticker=ticker,
                     label_standoff=2, 
                     border_line_color=None, 
                     location=(0,0),
                     major_label_text_font = 'nunito')

p.add_layout(color_bar, 'right')

hover= HoverTool(tooltips = [
                            ("Median Rent as a Percentage of Household Income","@color")
                            ]
                )

p.add_tools(hover)

show(p)
export_png(p, 'output/chloropeth_map.png')

<img svc="output/chloropeth_map.png">

<img src="output/chloropeth_map.png">